In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
from sklearn import metrics
from imblearn.over_sampling import SMOTE

In [2]:
#load data
train = pd.read_csv('C:/Stats Consulting/train_final.csv')
train.head()

,user_id,seller_id,label,user_gender_female,user_gender_male,user_gender_unknown,user_age_0_18,user_age_18_24,user_age_25_29,user_age_30_34,...,user_seller_add_to_favorite_cnt_db11_mth,user_seller_click_rt_db11,user_seller_purchase_rt_db11,user_seller_add_to_favorite_rt_db11,user_seller_click_rt_db11_wk,user_seller_purchase_rt_db11_wk,user_seller_add_to_favorite_rt_db11_wk,user_seller_click_rt_db11_mth,user_seller_purchase_rt_db11_mth,user_seller_add_to_favorite_rt_db11_mth
0,34176,3906,0,1,0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,34176,121,0,1,0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,34176,4356,1,1,0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,34176,2217,0,1,0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,230784,4818,0,1,0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
test = pd.read_csv('C:/Stats Consulting/test_final.csv')
test.head()

,user_id,seller_id,user_gender_female,user_gender_male,user_gender_unknown,user_age_0_18,user_age_18_24,user_age_25_29,user_age_30_34,user_age_35_39,...,user_seller_add_to_favorite_cnt_db11_mth,user_seller_click_rt_db11,user_seller_purchase_rt_db11,user_seller_add_to_favorite_rt_db11,user_seller_click_rt_db11_wk,user_seller_purchase_rt_db11_wk,user_seller_add_to_favorite_rt_db11_wk,user_seller_click_rt_db11_mth,user_seller_purchase_rt_db11_mth,user_seller_add_to_favorite_rt_db11_mth
0,163968,4605,1,0,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,360576,1581,0,0,1,0.0,1.0,0.0,0.0,0.0,...,0.0,1.4,0.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,98688,1964,1,0,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,98688,3645,1,0,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,295296,3361,0,1,0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
#fill all infinite entry with 0
train = train.replace(np.inf, 0)
test = test.replace(np.inf, 0)

In [5]:
#data split (training_set, testing_set)
data_output = train.label
data_input = train.drop('label',axis=1)
#unbalanced
data_output.sum()/data_output.count()

0.06115063788027478

In [6]:
training_set_x, testing_set_x, training_set_y, testing_set_y = train_test_split(data_input, data_output, test_size=0.3, random_state=1)
print(training_set_x.shape, testing_set_x.shape, training_set_y.shape, testing_set_y.shape)

(182604, 81) (78260, 81) (182604,) (78260,)


In [7]:
np.isfinite(training_set_x).all()

user_id                                    True
seller_id                                  True
user_gender_female                         True
user_gender_male                           True
user_gender_unknown                        True
                                           ... 
user_seller_purchase_rt_db11_wk            True
user_seller_add_to_favorite_rt_db11_wk     True
user_seller_click_rt_db11_mth              True
user_seller_purchase_rt_db11_mth           True
user_seller_add_to_favorite_rt_db11_mth    True
Length: 81, dtype: bool

In [28]:
#initial model fitting
#random forest
model_rf = RandomForestClassifier(n_estimators=100, criterion='gini',bootstrap= True,max_features ='sqrt')
#fit on training data
model_rf.fit(training_set_x, training_set_y)
#predictions
rf_predictions = model_rf.predict(testing_set_x)
rf_probs = model_rf.predict_proba(testing_set_x)[:, 1]
#check model accuracy, ROC AUC
print('Accuracy:',metrics.accuracy_score(testing_set_y, rf_predictions))
print('ROC AUC:', metrics.roc_auc_score(testing_set_y, rf_probs))

Accuracy: 0.9381165346281626
ROC AUC: 0.6282399001136609


In [29]:
np.mean(rf_predictions)

0.0001277791975466394

In [9]:
#Gradient Boosting
model_gbm = lgb.LGBMClassifier(objective='binary', metric='binary_logloss',learning_rate=0.03)
#fit on training data
model_gbm.fit(training_set_x, training_set_y)
#predictions
gbm_predictions = model_gbm.predict(testing_set_x)
gbm_probs = model_gbm.predict_proba(testing_set_x)[:, 1]
#check model accuracy, ROC AUC
print('Accuracy:', metrics.accuracy_score(testing_set_y, gbm_predictions))
print('ROC AUC', metrics.roc_auc_score(testing_set_y, gbm_probs))

Accuracy: 0.9381165346281626
ROC AUC 0.6608758699703369


In [23]:
np.mean(gbm_predictions)

0.0

In [16]:
#SMOTE
sm = SMOTE(random_state=2)
training_set_x_res, training_set_y_res = sm.fit_sample(training_set_x, training_set_y.ravel())
#print percentage of output after oversampling
print('After oversampling, counts of label 1:',format(sum(training_set_y_res==1)))
print('After oversampling, counts of label 0:',format(sum(training_set_y_res==0)))

After oversampling, counts of label 1: 171495
After oversampling, counts of label 0: 171495


In [17]:
#random forest model fitting with over sampling data
model_rf_2 = RandomForestClassifier(n_estimators=100, criterion='gini',bootstrap= True,max_features ='sqrt')
#fit on training data
model_rf_2.fit(training_set_x_res, training_set_y_res)
#predictions
rf2_predictions = model_rf_2.predict(testing_set_x)
rf2_probs = model_rf_2.predict_proba(testing_set_x)[:, 1]
#check model accuracy, ROC AUC
print('Accuracy:',metrics.accuracy_score(testing_set_y, rf2_predictions))
print('ROC AUC:', metrics.roc_auc_score(testing_set_y, rf2_probs))

Accuracy: 0.9375159723996933
ROC AUC: 0.6175108213055363


In [24]:
np.mean(rf2_predictions)

0.0011116790186557629

In [18]:
#Gradient Boosting
model_gbm_2 = lgb.LGBMClassifier(objective='binary', metric='binary_logloss',learning_rate=0.03)
#fit on training data
model_gbm_2.fit(training_set_x_res, training_set_y_res)
#predictions
gbm2_predictions = model_gbm_2.predict(testing_set_x)
gbm2_probs = model_gbm_2.predict_proba(testing_set_x)[:, 1]
#check model accuracy, ROC AUC
print('Accuracy:', metrics.accuracy_score(testing_set_y, gbm2_predictions))
print('ROC AUC', metrics.roc_auc_score(testing_set_y, gbm2_probs))

Accuracy: 0.9380782008688986
ROC AUC 0.6003051801336191


In [25]:
np.mean(gbm2_predictions)

0.00011500127779197546